ISR Approach 1 get policy response 

In [103]:
import os,sys # ,getpass
sys.path.insert(0,'../../libs')
import pandas as pd 
from utils import load_json, extract_json_string,get_all_files
from oai_ast_utils import OpenAIAssistant_Base
import tqdm,json,time
#from openai import OpenAI
## load API Key
key = load_json('/root/workspace/key/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT1']['API_KEY']
#os.environ['OPENAI_API_KEY'] = key['ISR']['API_KEY']

In [104]:
ISRBot = OpenAIAssistant_Base()
ISRBot.FileManager._get_file_info()


,id,bytes,created_at,filename,object,purpose,status,status_details
0,file-USdjzVAJgclWyxxtYc4tIiUO,3466567,1704572287,USA_2022.pdf,file,assistants,processed,None
1,file-fRXHBzeSfTYhcu82NfZ5MlL8,3966712,1704572286,UAE_2021.pdf,file,assistants,processed,None
2,file-SEKf1SIkmnacDs70Cpfodz5j,2540360,1704572285,Tanzania_2023.pdf,file,assistants,processed,None
3,file-8QD58MFCIiObTdKB2U5hh5ON,7302895,1704572283,Indonesia_2023.pdf,file,assistants,processed,None
4,file-qWu8glM3tmEn3ymRwbRO3WXo,2814763,1704572281,Belgium_2022.pdf,file,assistants,processed,None


In [105]:
## load file 
res_folder = '/root/workspace/data/ISR_Results'
PDF_folder = '/root/workspace/data/DOCs/PDF'
results = load_json(os.path.join(res_folder,'raw_msg.json'))


- first check see if all needed files are there 

In [106]:
file_names = list(results.keys())
print('needed files : {}'.format(file_names))
server_file_info = ISRBot.FileManager.get_files_info_by(filter_criteria={'filename':file_names},return_fields=['filename'])
for fn in file_names:
    if fn not in server_file_info:
        fp = os.path.join(PDF_folder,fn)
        fid,fname = ISRBot.FileManager.upload_file(fp,purpose='assistants')
        print('uploaded {}'.format(fname))

needed files : ['USA_2022.pdf', 'UAE_2021.pdf', 'Tanzania_2023.pdf', 'Indonesia_2023.pdf', 'Belgium_2022.pdf']


- get all files need to be processed 

In [107]:
loop_files = ISRBot.FileManager.get_files_info_by(filter_criteria={'filename':file_names},return_fields=['id','filename'],to_dict=True)
print(loop_files)

[{'id': 'file-USdjzVAJgclWyxxtYc4tIiUO', 'filename': 'USA_2022.pdf'}, {'id': 'file-fRXHBzeSfTYhcu82NfZ5MlL8', 'filename': 'UAE_2021.pdf'}, {'id': 'file-SEKf1SIkmnacDs70Cpfodz5j', 'filename': 'Tanzania_2023.pdf'}, {'id': 'file-8QD58MFCIiObTdKB2U5hh5ON', 'filename': 'Indonesia_2023.pdf'}, {'id': 'file-qWu8glM3tmEn3ymRwbRO3WXo', 'filename': 'Belgium_2022.pdf'}]


- process raw mesg and formulate prompt

In [108]:
processed_res = {k:json.loads(extract_json_string(v['msg'])) for k,v in results.items()}

In [109]:
# list_of_dicts=processed_res['USA_2022.pdf']['risks']
# def dict2string(list_of_dicts):
#     result_string = ""
#     for d in list_of_dicts:
#         for key, value in d.items():
#             result_string += f"{key}: {value}, "
#         result_string = result_string.rstrip(', ') + '; \n'
#     return result_string

In [111]:
prompt_identify_policy = """
You are an economist in the IMF. Your main task is to review Country Staff reports.
Please read carefully the document and the listed macro critical risk and its description. 
From the document, please identify all suggested policy responses to mitigate this specific risk listed. 
Please make sure you only provide policy responses directly targeted at this specific risk. 
Please only use content in the provided document and nothing else.

The identified risks below:

----------

{RISK_ITEM}

----------

Output is a JSON object with the following format: {{"risk_name": "<risk_name>", "risk_description": "<risk_description>", "policy_responses":["<policy response1>","<policy response2>", ...... ]}}

Please proceed with the task, keeping in mind the importance of accuracy and clarity in your analysis.

"""

- print one example of prompts

In [112]:
list_of_dicts=processed_res['USA_2022.pdf']['risks']
risk_items = ["{}: {}".format(i.get('risk_name'),i.get('risk_description')) for i in list_of_dicts]
print(prompt_identify_policy.format(RISK_ITEM = risk_items[0]))


You are an economist in the IMF. Your main task is to review Country Staff reports.
Please read carefully the document and the listed macro critical risk and its description. 
From the document, please identify all suggested policy responses to mitigate this specific risk listed. 
Please make sure you only provide policy responses directly targeted at this specific risk. 
Please only use content in the provided document and nothing else.

The identified risks below:

----------

Russia’s Invasion of Ukraine: Escalation of sanctions and other disruptions due to Russia’s invasion of Ukraine could lead to even higher commodity prices, refugee migration, tighter financial conditions, and other adverse spillovers that particularly affect Low-Income Countries (LICs) and commodity-importing Emerging Markets (EMs)【9†source】.

----------

Output is a JSON object with the following format: {"risk_name": "<risk_name>", "risk_description": "<risk_description>", "policy_responses":["<policy respon

In [113]:
CREATE_AST = False
if CREATE_AST: 
    ISRBot.create_assistant(name='ISR Bot',
                        description='This bot is created to help ISR project',
                        instructions="You are an economist in the IMF. Your main task is to review Country Staff reports.")
    print(ISRBot.assistant.name , ISRBot.assistant.id)

In [114]:
## reload assistant ; sometime there are bugs if keep reusing the same assistant
filter_criteria={'name':['ISR Bot']}
as_id = ISRBot.get_assistants_info_by(filter_criteria,return_fields=['id'])
print(as_id)
## retrieve by id 
assit = ISRBot.get_assistants_by_ids(as_id[0])
print(assit.name, assit.id )
## set the retrieved assistant as current active assistant 
ISRBot._set_active_assistant(assit)

['asst_4oWbFLPVW77u9qbxFcLSlklb']
ISR Bot asst_4oWbFLPVW77u9qbxFcLSlklb
set ISR Bot as current active assistant.


- Now run policy identification 

In [116]:
results_policy={}
for f_dict in loop_files:
    # f_dict = loop_files[0]
    fid,fname = f_dict.get('id'),f_dict.get('filename')
    ass_info_dict = {
                'model':"gpt-4-1106-preview",
                'file_ids':[fid] ### USA file id
                }
    ISRBot.update_current_assistant(**ass_info_dict)
    
    ## run the query 
    results_policy[fname] = []
    list_of_dicts=processed_res[fname]['risks']
    risk_items = ["{}: {}".format(i.get('risk_name'),i.get('risk_description')) for i in list_of_dicts]
    
    for ri in risk_items:
        user_input_dict = {"role":"user",
                        "content":prompt_identify_policy.format(RISK_ITEM = ri)
                        }
        msg,status = ISRBot.quick_query(user_input_dict)
        results_policy[fname].append({'msg':msg,'status':status})
        

Elapsed time: 19.46 s || Status: completed      

In [117]:
### export raw policy response to json 
with open(os.path.join(res_folder,'raw_policy_response_msg.json'), 'w') as file:
    json.dump(results_policy, file, indent=4)
    
#### post process raw results 
processed_policy_res = {}
for fn,vs in results_policy.items():
    processed_policy_res[fn]=[]
    for v in vs:
        try:
            temp_res = json.loads(extract_json_string(v['msg']))
            processed_policy_res[fn].append(temp_res)
        except:
            print('error {}'.format(v))
        

In [118]:
policy_dfs = {}
excel_out_path = os.path.join(res_folder,'policy.xlsx')
with pd.ExcelWriter(excel_out_path, engine='openpyxl') as writer:
    for k,v in processed_policy_res.items():
        t_df = pd.DataFrame(v)
        expanded_df = t_df.explode('policy_responses')
        expanded_df.to_excel(writer, sheet_name=k)
